# train

> Perform optimization using DiffPASS models

In [ ]:
#| default_exp train

In [ ]:
#| export

# Stdlib imports
from collections.abc import Iterable, Sequence
from typing import Optional, Union, Any, Literal
from dataclasses import dataclass
from copy import deepcopy

# Progress bars
from tqdm import tqdm

# PyTorch
import torch
from torch.nn import Module

# DiffPASS imports
from diffpass.base import EnsembleMixin, DiffPASSMixin, scalar_or_1d_tensor
from diffpass.model import (
    GeneralizedPermutation,
    EnsembleMatrixApply,
    TwoBodyEntropyLoss,
    MILoss,
    HammingSimilarities,
    BestHits,
    Blosum62Similarities,
    InterGroupLoss,
    IntraGroupLoss,
)

In [ ]:
#| export

def global_argmax_from_group_argmaxes(mats: Iterable[torch.Tensor]) -> torch.Tensor:
    global_argmax = []
    start_idx = 0
    for mats_this_group in mats:
        global_argmax.append(mats_this_group.argmax(-1) + start_idx)
        start_idx += mats_this_group.shape[-1]

    return torch.cat(global_argmax, dim=-1)


def apply_hard_permutation_batch_to_similarity(
    *, x: torch.Tensor, perms: list[torch.Tensor]
) -> torch.Tensor:
    """
    Conjugate a single similarity matrix by a batch of hard permutations.

    Args:
        perms: List of batches of permutation matrices of shape (..., D, D).
        x: Similarity matrix of shape (D, D).

    Returns:
        Batch of conjugated matrices of shape (..., D, D).
    """
    global_argmax = global_argmax_from_group_argmaxes(perms)
    x_permuted_rows = x[global_argmax]

    # Permuting columns is more involved
    index = global_argmax.view(*global_argmax.shape[:-1], 1, -1).expand(
        *global_argmax.shape, global_argmax.shape[-1]
    )
    # Example of gather with 4D tensor and dim=-1:
    # out[i][j][k][l] = input[i][j][k][index[i][j][k][l]]

    return torch.gather(x_permuted_rows, -1, index)


def _dcc(x: torch.Tensor) -> torch.Tensor:
    return x.detach().clone().cpu()

In [ ]:
def test_batch_perm(shape: tuple[int, int, int, int]):
    perms = torch.randn(*shape)
    x = torch.randn(shape[-2], shape[-1])

    argmax = perms.argmax(-1)
    x_permuted_rows = x[argmax]
    index = argmax.view(*argmax.shape[:-1], 1, -1).expand_as(perms)
    output = torch.gather(x_permuted_rows, -1, index)

    expected = torch.stack([
        torch.stack([
            x[argmax[i, j], :][:, argmax[i, j]] for j in range(shape[1])
        ], dim=0) for i in range(shape[0])
    ], dim=0)

    assert torch.equal(output, expected)


test_batch_perm((2, 5, 4, 4))

In [ ]:
#| export

@dataclass
class DiffPASSResults:
    log_alphas: list[list[torch.Tensor]]
    # Perms
    soft_perms: list[list[torch.Tensor]]
    hard_perms: list[list[torch.Tensor]]
    # Losses
    hard_losses: dict[str, list[torch.Tensor]]
    soft_losses: dict[str, list[torch.Tensor]]
    # Losses with identity perm
    hard_losses_identity_perm: dict[str, Optional[float]]
    soft_losses_identity_perm: Optional[dict[str, Optional[float]]] = None

In [ ]:
#| export

class Information(Module, EnsembleMixin, DiffPASSMixin):
    def __init__(
        self,
        group_sizes: Iterable[int],
        fixed_matchings: Optional[Sequence[Sequence[Sequence[int]]]] = None,
        permutation_cfg: Optional[dict[str, Any]] = None,
        information_measure: Literal["MI", "TwoBodyEntropy"] = "TwoBodyEntropy",
    ):
        super().__init__()
        self.group_sizes = tuple(s for s in group_sizes)
        self.fixed_matchings = fixed_matchings
        self.permutation_cfg = permutation_cfg
        self.information_measure = information_measure

        ensemble_shape = []
        _dim_in_ensemble = -1

        if permutation_cfg is None:
            permutation_cfg = {}
        else:
            self.validate_permutation_cfg(permutation_cfg)
            permutation_cfg = deepcopy(permutation_cfg)
            _dim_in_ensemble = self._adjust_cfg_and_ensemble_shape(
                ensemble_shape=ensemble_shape,
                cfg=permutation_cfg,
                param_name="tau",
                prev_dim_in_ensemble=_dim_in_ensemble,
            )
        self.effective_permutation_cfg_ = permutation_cfg
        ensemble_shape = tuple(ensemble_shape)

        self.validate_information_measure(information_measure)

        self.permutation = GeneralizedPermutation(
            group_sizes=self.group_sizes,
            ensemble_shape=ensemble_shape,
            fixed_matchings=fixed_matchings,
            mode="soft",
            **permutation_cfg,
        )
        self.ensemble_matrix_apply = EnsembleMatrixApply(group_sizes=self.group_sizes)
        if self.information_measure == "TwoBodyEntropy":
            self.information_loss = TwoBodyEntropyLoss()
        elif self.information_measure == "MI":
            self.information_loss = MILoss()

    @staticmethod
    def _adjust_cfg_and_ensemble_shape(
        *,
        ensemble_shape: list[int],
        cfg: dict[str, Any],
        param_name: str,
        prev_dim_in_ensemble: int = -1,
    ) -> int:
        new_dim_in_ensemble = prev_dim_in_ensemble
        if param_name in cfg:
            param = scalar_or_1d_tensor(param=cfg[param_name], param_name=param_name)
            cfg[param_name] = param
            if param.ndim == 1:
                ensemble_shape += list(param.shape)
                new_dim_in_ensemble += 1
                cfg[f"{param_name}_dim_in_ensemble"] = new_dim_in_ensemble

        return new_dim_in_ensemble

    def forward(
        self,
        x: torch.Tensor,
        y: torch.Tensor,
    ) -> dict[str, torch.Tensor]:
        mode = self.permutation.mode

        # Soft or hard permutations (list)
        perms = self.permutation()
        x_perm = self.ensemble_matrix_apply(x, mats=perms)

        # Two-body entropy portion of the loss
        loss_info = self.information_loss(x_perm, y)

        return {"perms": perms, "x_perm": x_perm, "loss_info": loss_info}

    def soft_(self) -> None:
        self.permutation.soft_()

    def hard_(self) -> None:
        self.permutation.hard_()

    def _prepare_fit(self, x: torch.Tensor, y: torch.Tensor) -> DiffPASSResults:
        # Validate inputs
        self.validate_inputs(x, y)

        # Initialize DiffPassResults object
        results = DiffPASSResults(
            log_alphas=[],
            soft_perms=[],
            hard_perms=[],
            soft_losses={self.information_measure: []},
            hard_losses={self.information_measure: []},
            hard_losses_identity_perm={
                self.information_measure: None,
            },
            soft_losses_identity_perm={
                self.information_measure: None,
            },
        )

        # Compute hard losses with identity permutation
        self.hard_()
        with torch.no_grad():
            results.hard_losses_identity_perm[
                self.information_measure
            ] = self.information_loss(x, y).item()
            results.soft_losses_identity_perm[
                self.information_measure
            ] = results.hard_losses_identity_perm[self.information_measure]

        return results

    def _fit(
        self,
        x: torch.Tensor,
        y: torch.Tensor,
        results: DiffPASSResults,
        *,
        epochs: int = 1,
        optimizer_name: Optional[str] = "SGD",
        optimizer_kwargs: Optional[dict[str, Any]] = None,
        mean_centering: bool = True,
        show_pbar: bool = True,
        compute_final_soft: bool = True,
    ) -> DiffPASSResults:
        # Initialize optimizer
        optimizer_cls = getattr(torch.optim, optimizer_name)
        optimizer_kwargs = (
            {"lr": 1e-1} if optimizer_kwargs is None else deepcopy(optimizer_kwargs)
        )
        self.optimizer_ = optimizer_cls(self.parameters(), **optimizer_kwargs)

        # Progress bar
        pbar = range(epochs + 1)
        if show_pbar:
            pbar = tqdm(pbar)

        # ------------------------------------------------------------------------------------------
        ## Gradient descent
        # ------------------------------------------------------------------------------------------
        with torch.set_grad_enabled(True):
            self.optimizer_.zero_grad()
            for i in pbar:
                # Hard pass
                self.hard_()
                with torch.no_grad():
                    epoch_results = self(x, y)
                    loss_info = epoch_results["loss_info"]
                    perms = epoch_results["perms"]
                    results.log_alphas.append(
                        [_dcc(log_alpha) for log_alpha in self.permutation.log_alphas]
                    )
                    results.hard_perms.append(
                        [
                            _dcc(perms_this_group).argmax(-1).to(torch.int16)
                            for perms_this_group in perms
                        ]
                    )
                    results.hard_losses[self.information_measure].append(
                        _dcc(loss_info)
                    )

                # Soft pass
                if i < epochs or compute_final_soft:
                    self.soft_()
                    epoch_results = self(x, y)
                    loss_info = epoch_results["loss_info"]
                    perms = epoch_results["perms"]
                    results.soft_perms.append(
                        [_dcc(perms_this_group) for perms_this_group in perms]
                    )
                    results.soft_losses[self.information_measure].append(
                        _dcc(loss_info)
                    )

                    loss = loss_info.sum()
                    loss.backward()
                if i < epochs:
                    self.optimizer_.step()
                    self.optimizer_.zero_grad()
                    if mean_centering:
                        with torch.no_grad():
                            for log_alpha in self.permutation.log_alphas:
                                log_alpha[...] -= log_alpha.mean(
                                    dim=(-1, -2), keepdim=True
                                )

        return results

    def fit(
        self,
        x: torch.Tensor,
        y: torch.Tensor,
        *,
        epochs: int = 1,
        optimizer_name: Optional[str] = "SGD",
        optimizer_kwargs: Optional[dict[str, Any]] = None,
        mean_centering: bool = True,
        show_pbar: bool = True,
        compute_final_soft: bool = True,
    ) -> DiffPASSResults:
        results = self._prepare_fit(x, y)
        results = self._fit(
            x,
            y,
            results,
            epochs=epochs,
            optimizer_name=optimizer_name,
            optimizer_kwargs=optimizer_kwargs,
            mean_centering=mean_centering,
            show_pbar=show_pbar,
            compute_final_soft=compute_final_soft,
        )

        return results

In [ ]:
#| export

class InformationAndBestHits(Module, EnsembleMixin, DiffPASSMixin):
    def __init__(
        self,
        group_sizes: Iterable[int],
        fixed_matchings: Optional[Sequence[Sequence[Sequence[int]]]] = None,
        loss_weights: Optional[dict[str, Union[float, torch.Tensor]]] = None,
        permutation_cfg: Optional[dict[str, Any]] = None,
        information_measure: Literal["MI", "TwoBodyEntropy"] = "TwoBodyEntropy",
        similarity_kind: Literal["Hamming", "Blosum62"] = "Hamming",
        similarities_cfg: Optional[dict[str, Any]] = None,
        best_hits_cfg: Optional[dict[str, Any]] = None,
        inter_group_loss_score_fn: Optional[callable] = None,
    ):
        super().__init__()
        self.group_sizes = tuple(s for s in group_sizes)
        self.fixed_matchings = fixed_matchings
        self.loss_weights = loss_weights
        self.permutation_cfg = permutation_cfg
        self.information_measure = information_measure
        self.similarity_kind = similarity_kind
        self.similarities_cfg = similarities_cfg
        self.best_hits_cfg = best_hits_cfg
        self.inter_group_loss_score_fn = inter_group_loss_score_fn

        ensemble_shape = []
        _dim_in_ensemble = -1

        if permutation_cfg is None:
            permutation_cfg = {}
        else:
            self.validate_permutation_cfg(permutation_cfg)
            permutation_cfg = deepcopy(permutation_cfg)
            _dim_in_ensemble = self._adjust_cfg_and_ensemble_shape(
                ensemble_shape=ensemble_shape,
                cfg=permutation_cfg,
                param_name="tau",
                prev_dim_in_ensemble=_dim_in_ensemble,
            )
        self.effective_permutation_cfg_ = permutation_cfg

        self.validate_information_measure(information_measure)
        self.loss_weights_keys = {self.information_measure, "BestHits"}

        self.validate_similarity_kind(similarity_kind)
        if similarities_cfg is None:
            similarities_cfg = {}
        else:
            self.validate_similarities_cfg(similarities_cfg)
            similarities_cfg = deepcopy(similarities_cfg)
        self.effective_similarities_cfg_ = similarities_cfg

        if best_hits_cfg is None:
            best_hits_cfg = {}
        else:
            self.validate_best_hits_cfg(best_hits_cfg)
            best_hits_cfg = deepcopy(best_hits_cfg)
            _dim_in_ensemble = self._adjust_cfg_and_ensemble_shape(
                ensemble_shape=ensemble_shape,
                cfg=best_hits_cfg,
                param_name="tau",
                prev_dim_in_ensemble=_dim_in_ensemble,
            )
        self.effective_best_hits_cfg_ = best_hits_cfg

        if loss_weights is not None:
            loss_weights = deepcopy(loss_weights)
            _dim_in_ensemble = self._adjust_loss_weights_and_ensemble_shape(
                loss_weights=loss_weights,
                ensemble_shape=ensemble_shape,
                prev_dim_in_ensemble=_dim_in_ensemble,
            )
        else:
            loss_weights = {
                self.information_measure: torch.tensor(0.5),
                "BestHits": torch.tensor(0.5),
            }

        ensemble_shape = tuple(ensemble_shape)

        self.permutation = GeneralizedPermutation(
            group_sizes=self.group_sizes,
            ensemble_shape=ensemble_shape,
            fixed_matchings=fixed_matchings,
            mode="soft",
            **permutation_cfg,
        )
        self.ensemble_matrix_apply = EnsembleMatrixApply(group_sizes=self.group_sizes)
        if self.information_measure == "TwoBodyEntropy":
            self.information_loss = TwoBodyEntropyLoss()
        elif self.information_measure == "MI":
            self.information_loss = MILoss()
        self.information_loss.register_buffer(
            "weight", loss_weights[self.information_measure]
        )

        if similarity_kind == "Blosum62":
            self.similarities = Blosum62Similarities(**similarities_cfg)
        elif similarity_kind == "Hamming":
            self.similarities = HammingSimilarities(**similarities_cfg)
        self.best_hits = BestHits(
            group_sizes=self.group_sizes,
            ensemble_shape=ensemble_shape,
            mode="soft",
            **best_hits_cfg,
        )
        self.best_hits.register_buffer("weight", loss_weights["BestHits"])
        self.inter_group_loss = InterGroupLoss(
            group_sizes=self.group_sizes, score_fn=self.inter_group_loss_score_fn
        )

    @property
    def effective_loss_weights_(self) -> dict[str, torch.Tensor]:
        return {
            self.information_measure: self.information_loss.weight,
            "BestHits": self.best_hits.weight,
        }

    @staticmethod
    def _adjust_cfg_and_ensemble_shape(
        *,
        ensemble_shape: list[int],
        cfg: dict[str, Any],
        param_name: str,
        prev_dim_in_ensemble: int = -1,
    ) -> int:
        new_dim_in_ensemble = prev_dim_in_ensemble
        if param_name in cfg:
            param = scalar_or_1d_tensor(param=cfg[param_name], param_name=param_name)
            cfg[param_name] = param
            if param.ndim == 1:
                ensemble_shape += list(param.shape)
                new_dim_in_ensemble += 1
                cfg[f"{param_name}_dim_in_ensemble"] = new_dim_in_ensemble

        return new_dim_in_ensemble

    def _adjust_loss_weights_and_ensemble_shape(
        self,
        *,
        loss_weights: Optional[dict[str, Any]] = None,
        ensemble_shape: list[int],
        prev_dim_in_ensemble: int = -1,
    ) -> int:
        new_dim_in_ensemble = prev_dim_in_ensemble
        if set(loss_weights) != self.loss_weights_keys:
            raise ValueError(
                "`loss_weights` must be a dictionary with key set equal to "
                f"{self.loss_weights_keys}."
            )

        loss_weight_entr = loss_weights[self.information_measure]
        loss_weight_entr = scalar_or_1d_tensor(
            param=loss_weight_entr,
            param_name=f"``loss_weights['{self.information_measure}']``",
        )
        loss_weight_bh = loss_weights["BestHits"]
        loss_weight_bh = scalar_or_1d_tensor(
            param=loss_weight_bh, param_name="``loss_weights['BestHits']``"
        )

        if loss_weight_entr.ndim == 1 and loss_weight_bh.ndim == 1:
            if loss_weight_entr.shape != loss_weight_bh.shape:
                raise ValueError(
                    f"1D ``loss_weights['{self.information_measure}']`` and 1D "
                    "``loss_weights['BestHits']`` must have the same shape."
                )
            ensemble_shape += list(loss_weight_entr.shape)
            new_dim_in_ensemble += 1
        elif loss_weight_entr.ndim == 1 and loss_weight_bh.ndim == 0:
            loss_weight_bh = loss_weight_bh.repeat(loss_weight_entr.shape[0])
            ensemble_shape += list(loss_weight_entr.shape)
            new_dim_in_ensemble += 1
        elif loss_weight_entr.ndim == 0 and loss_weight_bh.ndim == 1:
            loss_weight_entr = loss_weight_entr.repeat(loss_weight_bh.shape[0])
            ensemble_shape += list(loss_weight_bh.shape)
            new_dim_in_ensemble += 1
        loss_weight_entr = self._reshape_ensemble_param(
            param=loss_weight_entr,
            ensemble_shape=ensemble_shape,
            dim_in_ensemble=new_dim_in_ensemble,
            n_dims_per_instance=0,
            param_name=f"``loss_weights['{self.information_measure}']``",
        )
        loss_weight_bh = self._reshape_ensemble_param(
            param=loss_weight_bh,
            ensemble_shape=ensemble_shape,
            dim_in_ensemble=new_dim_in_ensemble,
            n_dims_per_instance=0,
            param_name="``loss_weights['BestHits']``",
        )
        loss_weights[self.information_measure] = loss_weight_entr
        loss_weights["BestHits"] = loss_weight_bh

        return new_dim_in_ensemble

    def _precompute_bh(self, x: torch.Tensor, y: torch.Tensor) -> None:
        # Temporarily switch to hard BH
        self.best_hits.hard_()
        similarities_x = self.similarities(x)
        self.register_buffer("_bh_hard_x", self.best_hits(similarities_x))
        similarities_y = self.similarities(y)
        self.register_buffer("_bh_hard_y", self.best_hits(similarities_y))

        # Revert to soft (default) BH
        self.best_hits.soft_()
        self.register_buffer("_bh_soft_x", self.best_hits(similarities_x))
        similarities_y = self.best_hits.prepare_fixed(similarities_y)
        self.register_buffer("_bh_soft_y", self.best_hits(similarities_y))

    def forward(
        self,
        x: torch.Tensor,
        y: torch.Tensor,
        *,
        x_perm_hard: Optional[torch.Tensor] = None,
    ) -> dict[str, torch.Tensor]:
        mode = self.permutation.mode
        assert mode == self.best_hits.mode

        # Soft or hard permutations (list)
        perms = self.permutation()
        x_perm = self.ensemble_matrix_apply(x, mats=perms)

        # Two-body entropy portion of the loss
        loss_info = self.information_loss(x_perm, y)

        # Best hits portion of the loss, with shortcut for hard permutations
        if mode == "soft":
            if x_perm_hard is not None:
                x_perm = (x_perm_hard - x_perm).detach() + x_perm
            similarities_x = self.similarities(x_perm)
            bh_x = self.best_hits(similarities_x)
        else:
            bh_x = apply_hard_permutation_batch_to_similarity(
                x=self._bh_hard_x, perms=perms
            )
        # Ensure comparisons are only hard-hard or soft-soft
        loss_bh = self.inter_group_loss(bh_x, getattr(self, f"_bh_{mode}_y"))

        return {
            "perms": perms,
            "x_perm": x_perm,
            "loss_info": loss_info,
            "loss_bh": loss_bh,
        }

    def soft_(self) -> None:
        self.permutation.soft_()
        self.best_hits.soft_()

    def hard_(self) -> None:
        self.permutation.hard_()
        self.best_hits.hard_()

    def _prepare_fit(self, x: torch.Tensor, y: torch.Tensor) -> DiffPASSResults:
        # Validate inputs
        self.validate_inputs(x, y)

        # Precompute matrices of best hits
        self._precompute_bh(x, y)

        # Initialize DiffPassResults object
        results = DiffPASSResults(
            log_alphas=[],
            soft_perms=[],
            hard_perms=[],
            soft_losses={self.information_measure: [], "BestHits": []},
            hard_losses={self.information_measure: [], "BestHits": []},
            hard_losses_identity_perm={
                self.information_measure: None,
                "BestHits": None,
            },
            soft_losses_identity_perm={
                self.information_measure: None,
                "BestHits": None,
            },
        )

        # Compute hard losses with identity permutation
        self.hard_()
        with torch.no_grad():
            results.hard_losses_identity_perm[
                self.information_measure
            ] = self.information_loss(x, y).item()
            results.hard_losses_identity_perm["BestHits"] = self.inter_group_loss(
                self._bh_hard_x, self._bh_hard_y
            ).item()
            results.soft_losses_identity_perm[
                self.information_measure
            ] = results.hard_losses_identity_perm[self.information_measure]
            results.soft_losses_identity_perm["BestHits"] = _dcc(
                self.inter_group_loss(self._bh_soft_x, self._bh_soft_y)
            )

        return results

    def _fit(
        self,
        x: torch.Tensor,
        y: torch.Tensor,
        results: DiffPASSResults,
        *,
        epochs: int = 1,
        optimizer_name: Optional[str] = "SGD",
        optimizer_kwargs: Optional[dict[str, Any]] = None,
        mean_centering: bool = True,
        similarity_gradient_bypass: bool = False,
        show_pbar: bool = True,
        compute_final_soft: bool = True,
    ) -> DiffPASSResults:
        # Initialize optimizer
        optimizer_cls = getattr(torch.optim, optimizer_name)
        optimizer_kwargs = (
            {"lr": 1e-1} if optimizer_kwargs is None else deepcopy(optimizer_kwargs)
        )
        self.optimizer_ = optimizer_cls(self.parameters(), **optimizer_kwargs)

        # Progress bar
        pbar = range(epochs + 1)
        if show_pbar:
            pbar = tqdm(pbar)

        # ------------------------------------------------------------------------------------------
        ## Gradient descent
        # ------------------------------------------------------------------------------------------
        x_perm_hard = None
        with torch.set_grad_enabled(True):
            self.optimizer_.zero_grad()
            for i in pbar:
                # Hard pass
                self.hard_()
                with torch.no_grad():
                    epoch_results = self(x, y)
                    loss_info = epoch_results["loss_info"]
                    loss_bh = epoch_results["loss_bh"]
                    if similarity_gradient_bypass:
                        x_perm_hard = epoch_results["x_perm"]
                    perms = epoch_results["perms"]
                    results.log_alphas.append(
                        [_dcc(log_alpha) for log_alpha in self.permutation.log_alphas]
                    )
                    results.hard_perms.append(
                        [
                            _dcc(perms_this_group).argmax(-1).to(torch.int16)
                            for perms_this_group in perms
                        ]
                    )
                    results.hard_losses[self.information_measure].append(
                        _dcc(loss_info)
                    )
                    results.hard_losses["BestHits"].append(_dcc(loss_bh))

                # Soft pass
                if i < epochs or compute_final_soft:
                    self.soft_()
                    epoch_results = self(x, y, x_perm_hard=x_perm_hard)
                    loss_info = epoch_results["loss_info"]
                    loss_bh = epoch_results["loss_bh"]
                    perms = epoch_results["perms"]
                    results.soft_perms.append(
                        [_dcc(perms_this_group) for perms_this_group in perms]
                    )
                    results.soft_losses[self.information_measure].append(
                        _dcc(loss_info)
                    )
                    results.soft_losses["BestHits"].append(_dcc(loss_bh))

                    loss = (
                        self.information_loss.weight * loss_info
                        + self.best_hits.weight * loss_bh
                    ).sum()
                    loss.backward()
                if i < epochs:
                    self.optimizer_.step()
                    self.optimizer_.zero_grad()
                    if mean_centering:
                        with torch.no_grad():
                            for log_alpha in self.permutation.log_alphas:
                                log_alpha[...] -= log_alpha.mean(
                                    dim=(-1, -2), keepdim=True
                                )

        return results

    def fit(
        self,
        x: torch.Tensor,
        y: torch.Tensor,
        *,
        epochs: int = 1,
        optimizer_name: Optional[str] = "SGD",
        optimizer_kwargs: Optional[dict[str, Any]] = None,
        mean_centering: bool = True,
        similarity_gradient_bypass: bool = False,
        show_pbar: bool = True,
        compute_final_soft: bool = True,
    ) -> DiffPASSResults:
        results = self._prepare_fit(x, y)
        results = self._fit(
            x,
            y,
            results,
            epochs=epochs,
            optimizer_name=optimizer_name,
            optimizer_kwargs=optimizer_kwargs,
            mean_centering=mean_centering,
            similarity_gradient_bypass=similarity_gradient_bypass,
            show_pbar=show_pbar,
            compute_final_soft=compute_final_soft,
        )

        return results

In [ ]:
#| export

class InformationAndMirrortree(Module, EnsembleMixin, DiffPASSMixin):
    def __init__(
        self,
        group_sizes: Iterable[int],
        fixed_matchings: Optional[Sequence[Sequence[Sequence[int]]]] = None,
        loss_weights: Optional[dict[str, Union[float, torch.Tensor]]] = None,
        permutation_cfg: Optional[dict[str, Any]] = None,
        information_measure: Literal["MI", "TwoBodyEntropy"] = "TwoBodyEntropy",
        similarity_kind: Literal["Hamming", "Blosum62"] = "Hamming",
        similarities_cfg: Optional[dict[str, Any]] = None,
        intra_group_loss_score_fn: Optional[callable] = None,
    ):
        super().__init__()
        self.group_sizes = tuple(s for s in group_sizes)
        self.fixed_matchings = fixed_matchings
        self.loss_weights = loss_weights
        self.permutation_cfg = permutation_cfg
        self.information_measure = information_measure
        self.similarity_kind = similarity_kind
        self.similarities_cfg = similarities_cfg
        self.intra_group_loss_score_fn = intra_group_loss_score_fn

        ensemble_shape = []
        _dim_in_ensemble = -1

        if permutation_cfg is None:
            permutation_cfg = {}
        else:
            self.validate_permutation_cfg(permutation_cfg)
            permutation_cfg = deepcopy(permutation_cfg)
            _dim_in_ensemble = self._adjust_cfg_and_ensemble_shape(
                ensemble_shape=ensemble_shape,
                cfg=permutation_cfg,
                param_name="tau",
                prev_dim_in_ensemble=_dim_in_ensemble,
            )
        self.effective_permutation_cfg_ = permutation_cfg

        self.validate_information_measure(information_measure)
        self.loss_weights_keys = {self.information_measure, "Mirrortree"}

        self.validate_similarity_kind(similarity_kind)
        if similarities_cfg is None:
            similarities_cfg = {}
        else:
            self.validate_similarities_cfg(similarities_cfg)
            similarities_cfg = deepcopy(similarities_cfg)
        self.effective_similarities_cfg_ = similarities_cfg

        if loss_weights is not None:
            loss_weights = deepcopy(loss_weights)
            _dim_in_ensemble = self._adjust_loss_weights_and_ensemble_shape(
                loss_weights=loss_weights,
                ensemble_shape=ensemble_shape,
                prev_dim_in_ensemble=_dim_in_ensemble,
            )
        else:
            loss_weights = {
                self.information_measure: torch.tensor(0.5),
                "Mirrortree": torch.tensor(0.5),
            }

        ensemble_shape = tuple(ensemble_shape)

        self.permutation = GeneralizedPermutation(
            group_sizes=self.group_sizes,
            ensemble_shape=ensemble_shape,
            fixed_matchings=fixed_matchings,
            mode="soft",
            **permutation_cfg,
        )
        self.ensemble_matrix_apply = EnsembleMatrixApply(group_sizes=self.group_sizes)
        if self.information_measure == "TwoBodyEntropy":
            self.information_loss = TwoBodyEntropyLoss()
        elif self.information_measure == "MI":
            self.information_loss = MILoss()
        self.information_loss.register_buffer(
            "weight", loss_weights[self.information_measure]
        )

        if similarity_kind == "Blosum62":
            self.similarities = Blosum62Similarities(**similarities_cfg)
        elif similarity_kind == "Hamming":
            self.similarities = HammingSimilarities(**similarities_cfg)
        self.intra_group_loss = IntraGroupLoss(
            group_sizes=self.group_sizes, score_fn=self.intra_group_loss_score_fn
        )
        self.intra_group_loss.register_buffer("weight", loss_weights["Mirrortree"])

    @property
    def effective_loss_weights_(self) -> dict[str, torch.Tensor]:
        return {
            self.information_measure: self.information_loss.weight,
            "Mirrortree": self.intra_group_loss.weight,
        }

    @staticmethod
    def _adjust_cfg_and_ensemble_shape(
        *,
        ensemble_shape: list[int],
        cfg: dict[str, Any],
        param_name: str,
        prev_dim_in_ensemble: int = -1,
    ) -> int:
        new_dim_in_ensemble = prev_dim_in_ensemble
        if param_name in cfg:
            param = scalar_or_1d_tensor(param=cfg[param_name], param_name=param_name)
            cfg[param_name] = param
            if param.ndim == 1:
                ensemble_shape += list(param.shape)
                new_dim_in_ensemble += 1
                cfg[f"{param_name}_dim_in_ensemble"] = new_dim_in_ensemble

        return new_dim_in_ensemble

    def _adjust_loss_weights_and_ensemble_shape(
        self,
        *,
        loss_weights: Optional[dict[str, Any]] = None,
        ensemble_shape: list[int],
        prev_dim_in_ensemble: int = -1,
    ) -> int:
        new_dim_in_ensemble = prev_dim_in_ensemble
        if set(loss_weights) != self.loss_weights_keys:
            raise ValueError(
                "`loss_weights` must be a dictionary with key set equal to "
                f"{self.loss_weights_keys}."
            )

        loss_weight_entr = loss_weights[self.information_measure]
        loss_weight_entr = scalar_or_1d_tensor(
            param=loss_weight_entr,
            param_name=f"``loss_weights['{self.information_measure}']``",
        )
        loss_weight_mt = loss_weights["Mirrortree"]
        loss_weight_mt = scalar_or_1d_tensor(
            param=loss_weight_mt, param_name="``loss_weights['Mirrortree']``"
        )

        if loss_weight_entr.ndim == 1 and loss_weight_mt.ndim == 1:
            if loss_weight_entr.shape != loss_weight_mt.shape:
                raise ValueError(
                    f"1D ``loss_weights['{self.information_measure}']`` and 1D "
                    "``loss_weights['Mirrortree']`` must have the same shape."
                )
            ensemble_shape += list(loss_weight_entr.shape)
            new_dim_in_ensemble += 1
        elif loss_weight_entr.ndim == 1 and loss_weight_mt.ndim == 0:
            loss_weight_mt = loss_weight_mt.repeat(loss_weight_entr.shape[0])
            ensemble_shape += list(loss_weight_entr.shape)
            new_dim_in_ensemble += 1
        elif loss_weight_entr.ndim == 0 and loss_weight_mt.ndim == 1:
            loss_weight_entr = loss_weight_entr.repeat(loss_weight_mt.shape[0])
            ensemble_shape += list(loss_weight_mt.shape)
            new_dim_in_ensemble += 1
        loss_weight_entr = self._reshape_ensemble_param(
            param=loss_weight_entr,
            ensemble_shape=ensemble_shape,
            dim_in_ensemble=new_dim_in_ensemble,
            n_dims_per_instance=0,
            param_name=f"``loss_weights['{self.information_measure}']``",
        )
        loss_weight_mt = self._reshape_ensemble_param(
            param=loss_weight_mt,
            ensemble_shape=ensemble_shape,
            dim_in_ensemble=new_dim_in_ensemble,
            n_dims_per_instance=0,
            param_name="``loss_weights['Mirrortree']``",
        )
        loss_weights[self.information_measure] = loss_weight_entr
        loss_weights["Mirrortree"] = loss_weight_mt

        return new_dim_in_ensemble

    def _precompute_similarities(self, x: torch.Tensor, y: torch.Tensor) -> None:
        self.register_buffer("_similarities_hard_x", self.similarities(x))
        self.register_buffer("_similarities_hard_y", self.similarities(y))

    def forward(
        self,
        x: torch.Tensor,
        y: torch.Tensor,
        *,
        x_perm_hard: Optional[torch.Tensor] = None,
    ) -> dict[str, torch.Tensor]:
        mode = self.permutation.mode

        # Soft or hard permutations (list)
        perms = self.permutation()
        x_perm = self.ensemble_matrix_apply(x, mats=perms)

        # Two-body entropy portion of the loss
        loss_info = self.information_loss(x_perm, y)

        # Mirrortree portion of the loss, with shortcut for hard permutations
        if mode == "soft":
            if x_perm_hard is not None:
                x_perm = (x_perm_hard - x_perm).detach() + x_perm
            similarities_x = self.similarities(x_perm)
        else:
            similarities_x = apply_hard_permutation_batch_to_similarity(
                x=self._similarities_hard_x, perms=perms
            )
        # Ensure comparisons are only hard-hard or soft-soft
        loss_mt = self.intra_group_loss(
            similarities_x, getattr(self, f"_similarities_hard_y")
        )

        return {
            "perms": perms,
            "x_perm": x_perm,
            "loss_info": loss_info,
            "loss_mt": loss_mt,
        }

    def soft_(self) -> None:
        self.permutation.soft_()

    def hard_(self) -> None:
        self.permutation.hard_()

    def _prepare_fit(self, x: torch.Tensor, y: torch.Tensor) -> DiffPASSResults:
        # Validate inputs
        self.validate_inputs(x, y)

        # Precompute matrices of Mirrortree
        self._precompute_similarities(x, y)

        # Initialize DiffPassResults object
        results = DiffPASSResults(
            log_alphas=[],
            soft_perms=[],
            hard_perms=[],
            soft_losses={self.information_measure: [], "Mirrortree": []},
            hard_losses={self.information_measure: [], "Mirrortree": []},
            hard_losses_identity_perm={
                self.information_measure: None,
                "Mirrortree": None,
            },
        )

        # Compute hard losses with identity permutation
        self.hard_()
        with torch.no_grad():
            results.hard_losses_identity_perm[
                self.information_measure
            ] = self.information_loss(x, y).item()
            results.hard_losses_identity_perm["Mirrortree"] = self.intra_group_loss(
                self._similarities_hard_x, self._similarities_hard_y
            ).item()

        return results

    def _fit(
        self,
        x: torch.Tensor,
        y: torch.Tensor,
        results: DiffPASSResults,
        *,
        epochs: int = 1,
        optimizer_name: Optional[str] = "SGD",
        optimizer_kwargs: Optional[dict[str, Any]] = None,
        mean_centering: bool = True,
        similarity_gradient_bypass: bool = False,
        show_pbar: bool = True,
        compute_final_soft: bool = True,
    ) -> DiffPASSResults:
        # Initialize optimizer
        optimizer_cls = getattr(torch.optim, optimizer_name)
        optimizer_kwargs = (
            {"lr": 1e-1} if optimizer_kwargs is None else deepcopy(optimizer_kwargs)
        )
        self.optimizer_ = optimizer_cls(self.parameters(), **optimizer_kwargs)

        # Progress bar
        pbar = range(epochs + 1)
        if show_pbar:
            pbar = tqdm(pbar)

        # ------------------------------------------------------------------------------------------
        ## Gradient descent
        # ------------------------------------------------------------------------------------------
        x_perm_hard = None
        with torch.set_grad_enabled(True):
            self.optimizer_.zero_grad()
            for i in pbar:
                # Hard pass
                self.hard_()
                with torch.no_grad():
                    epoch_results = self(x, y)
                    loss_info = epoch_results["loss_info"]
                    loss_mt = epoch_results["loss_mt"]
                    if similarity_gradient_bypass:
                        x_perm_hard = epoch_results["x_perm"]
                    perms = epoch_results["perms"]
                    results.log_alphas.append(
                        [_dcc(log_alpha) for log_alpha in self.permutation.log_alphas]
                    )
                    results.hard_perms.append(
                        [
                            _dcc(perms_this_group).argmax(-1).to(torch.int16)
                            for perms_this_group in perms
                        ]
                    )
                    results.hard_losses[self.information_measure].append(
                        _dcc(loss_info)
                    )
                    results.hard_losses["Mirrortree"].append(_dcc(loss_mt))

                # Soft pass
                if i < epochs or compute_final_soft:
                    self.soft_()
                    epoch_results = self(x, y, x_perm_hard=x_perm_hard)
                    loss_info = epoch_results["loss_info"]
                    loss_mt = epoch_results["loss_mt"]
                    perms = epoch_results["perms"]
                    results.soft_perms.append(
                        [_dcc(perms_this_group) for perms_this_group in perms]
                    )
                    results.soft_losses[self.information_measure].append(
                        _dcc(loss_info)
                    )
                    results.soft_losses["Mirrortree"].append(_dcc(loss_mt))

                    loss = (
                        self.information_loss.weight * loss_info
                        + self.intra_group_loss.weight * loss_mt
                    ).sum()
                    loss.backward()
                if i < epochs:
                    self.optimizer_.step()
                    self.optimizer_.zero_grad()
                    if mean_centering:
                        with torch.no_grad():
                            for log_alpha in self.permutation.log_alphas:
                                log_alpha[...] -= log_alpha.mean(
                                    dim=(-1, -2), keepdim=True
                                )

        return results

    def fit(
        self,
        x: torch.Tensor,
        y: torch.Tensor,
        *,
        epochs: int = 1,
        optimizer_name: Optional[str] = "SGD",
        optimizer_kwargs: Optional[dict[str, Any]] = None,
        mean_centering: bool = True,
        similarity_gradient_bypass: bool = False,
        show_pbar: bool = True,
        compute_final_soft: bool = True,
    ) -> DiffPASSResults:
        results = self._prepare_fit(x, y)
        results = self._fit(
            x,
            y,
            results,
            epochs=epochs,
            optimizer_name=optimizer_name,
            optimizer_kwargs=optimizer_kwargs,
            mean_centering=mean_centering,
            similarity_gradient_bypass=similarity_gradient_bypass,
            show_pbar=show_pbar,
            compute_final_soft=compute_final_soft,
        )

        return results

In [ ]:
#| export

def compute_num_correct_matchings(
    results: DiffPASSResults,
    *,
    index_based: bool,
    x_seqs: Optional[list[list[tuple]]] = None,
    y_seqs: Optional[list[list[tuple]]] = None,
    xy_seqs: Optional[list[dict[str, int]]] = None,
) -> list[int]:
    """Compute the number of matchings that are 'correct' at each step of the optimization.
    'Correct' means that they are present in the original paired MSAs, assumed to be the
    ground truth.

    If `index_based` is True, then the matchings are assumed to be represented as range indices
    for faster computation, and the `x_seqs`, `y_seqs`, and `xy_seqs` arguments are ignored.
    """
    correct = []
    if index_based:
        for perms in results.hard_perms:
            correct_this_step = 0
            for perm_this_group in perms:
                n_seqs_this_group = perm_this_group.shape[-1]
                correct_this_group = perm_this_group == torch.arange(n_seqs_this_group)
                correct_this_group = correct_this_group.sum().item()
                correct_this_step += correct_this_group
            correct.append(correct_this_step)
    else:
        for perms in results.hard_perms:
            correct_this_perm = 0
            for (
                perm_this_group,
                x_seqs_this_group,
                y_seqs_this_group,
                xy_seqs_this_group,
            ) in zip(perms, x_seqs, y_seqs, xy_seqs):
                _xy_seqs_this_group = xy_seqs_this_group.copy()
                x_seqs_this_group_perm = [
                    x_seqs_this_group[idx] for idx in perm_this_group
                ]
                for x_seq, y_seq in zip(x_seqs_this_group_perm, y_seqs_this_group):
                    xy_key = f"{x_seq}:{y_seq}"
                    if _xy_seqs_this_group.get(xy_key, 0) > 0:
                        _xy_seqs_this_group[xy_key] -= 1
                        correct_this_perm += 1

            correct.append(correct_this_perm)

    return correct